In [287]:
import pickle
import re

import psycopg2
import pandas as pd
from unidecode import unidecode

from local_settings import hostname, username, password, port, database

In [187]:
connection = psycopg2.connect(
            host=hostname, user=username,
            password=password, dbname=database,
            port=port
        )

In [188]:
cursor = connection.cursor()

consult = "SELECT * FROM data_row"
cursor.execute(consult)

results = cursor.fetchall()

cursor.close()

In [189]:
cursor = connection.cursor()

consult = """SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'data_row';"""
cursor.execute(consult)

columns = cursor.fetchall()

cursor.close()

In [190]:
connection.close()

In [191]:
print(results[0])

('Proyecto inmobilario en venta en Calle Coronel Odriozola 128, San Isidro, Lima', 'Calle Coronel Odriozola 128, San IsidroLima, Lima', 120375, 45, 2, 3, 'Departamento', 'Mozz se deriva de la palabra en italiano mozione, que significa movimiento, justamente porque el diseño, la ubicación del proyecto y sus características se adaptan a un estilo de vida dinámico que busca comodidad, practicidad y conectividad. <br><br>Mozz es un edificio de solamente 36 departamentos con áreas desde 45m2 que se adaptan a cada estilo de vida, teniendo incluso opciones flexibles en cuanto a número de habitaciones. <br><br>Está ubicado en una calle tranquila pero cercano a vías importantes de la ciudad que brindan la facilidad de movilizarse en auto, bicicleta, transporte público o a pie. El proyecto está rodeado de importantes centros empresariales, supermercados, tiendas, restaurantes, parques, bancos y muchos otros servicios que complementan las actividades diarias. <br><br>Además, cuenta con áreas comu

In [192]:
columns = [i[0] for i in columns]
columns

['title',
 'address',
 'price',
 'area',
 'bath',
 'room',
 'property_type',
 'description',
 'year',
 'parking',
 'near_cc',
 'near_school',
 'near_parks',
 'near_avenue',
 'security',
 'elevator',
 'rest_area',
 'pool',
 'web']

In [193]:
df = pd.DataFrame(results, columns=columns)

In [194]:
len(df)

26912

In [195]:
df.drop_duplicates(inplace=True)
len(df)

22807

In [196]:
def clean_text(text):
    cleaned = unidecode(text.lower())
    cleaned = ''.join(e for e in cleaned if e.isalnum() or e.isspace())

    return cleaned

In [198]:
df["clean_address"] = df["address"].apply(clean_text)
df["clean_address"]

0          calle coronel odriozola 128 san isidrolima lima
1          calle coronel odriozola 128 san isidrolima lima
2          calle coronel odriozola 128 san isidrolima lima
3          calle coronel odriozola 128 san isidrolima lima
4        jiron batallon callao norte   santa teresasant...
                               ...                        
26892    jiron inca ripac 197 jesus maria peru  jesus m...
26895        cnel gregorio albarracin lanchipa tacna tacna
26905    san juan de lurigancho 15427 peru  san juan de...
26906    parque iii  los gladiolos terreno ate peru  at...
26907    calle arturo duray 411 surquillo peru  surquil...
Name: clean_address, Length: 22807, dtype: object

In [199]:
dept = []

for i in df["clean_address"]:
    if re.search(r'\blima\b', i):
        dept.append('Lima')
    else:
        dept.append('')

df['dept'] = dept
df = df[df['dept'] != '']

In [258]:
districts = ['san juan de miraflores',
            'callao',
            'miraflores',
            'san isidro',
            'surco',
            'barranco',
            'chorrillos',
            'san borja',
            'lince',
            'la molina',
            'cercado de lima',
            'magdalena del mar',
            'carabayllo',
            'comas',
            'pueblo libre',
            'jesus maria',
            'la victoria',
            'los olivos',
            'lurigancho',
            'ate',
            'magdalena',
            'san martin',
            'san miguel',
            'surquillo',
            'san juan de lurigancho',
            'provincia de lima',
            'santiago de surco',
            'cercado',
            'limalima'
            ]

In [259]:
district = []

for ub in df["clean_address"]:
    dis = ""

    for d in districts:
        if re.search(f'.*{d}.*', ub):
            dis = d
            break

    if dis == "":
        print(ub)

    district.append(dis.title())


av las palmeras 110 san bartolo san bartololima lima
avenida principal 915 lima 15038 peru
avenida gerardo unger  el naranjalindependencia lima
av separadora industrial  avenida 200 millas pachacamaclima lima
jr loreto brenalima lima
jr lima 1824 la riverahuancayo huancayo
nueva toledo  cieneguillacieneguilla lima
jr lima 1824 la riverahuancayo huancayo
jr lima 1824 la riverahuancayo huancayo
avenida gerardo unger  el naranjalindependencia lima
jr lima 1824 la riverahuancayo huancayo
puente piedra puente piedralima lima
playa puerto viejo san antoniocanete lima
playa poseidon pucusanalima lima
parque del agustino los sauces block 9 departamento 502 el agustinoel agustino lima
malecon superior no 1858 lima peru
condominio farallones km 92 asiacanete lima
avenida juan velasco alvarado lima peru  lima lima lima
mala canete peru  mala canete lima
urb higuereta santa rosalima lima
avenida gerardo unger  el naranjalindependencia lima
avenida gerardo unger  el naranjalindependencia lima
aveni

In [266]:
for i in range(len(district)):
    dis = district[i]
    if dis == 'Limalima' or dis == 'Provincia De Lima' or dis == 'Cercado':
        district[i] = 'Cercado De Lima'
    if dis == 'San Juan De Lurigancho':
        district[i] = 'Lurigancho'
    if dis == 'Santiago De Surco':
        district[i] = 'Surco'
    if dis == 'Magdalena':
        district[i] = 'Magdalena Del Mar'
    if dis == 'San Juan De Miraflores':
        district[i] = 'Callao'


In [268]:
df["district"] = district
df = df[df["district"] != ""]
df["district"].value_counts()

/tmp/ipykernel_88492/4009375478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["district"] = district


district
Surco                3393
Miraflores           2227
San Isidro           1421
La Molina             899
San Borja             808
Surquillo             735
Cercado De Lima       732
San Miguel            667
Jesus Maria           554
Chorrillos            520
Barranco              494
Lince                 451
Los Olivos            420
Pueblo Libre          420
Magdalena Del Mar     418
Ate                   384
San Martin            367
Lurigancho            354
Carabayllo            319
Callao                299
La Victoria           223
Comas                 197
Name: count, dtype: int64

In [284]:
dfs = {}  # Creamos un diccionario para almacenar los DataFrames

for i in df["district"].value_counts().index.tolist():
    dfs[i] = pd.read_excel(f'test/{i.upper()}.xlsx')


In [285]:
for i in dfs:
    dfs[i] = round(dfs[i]["ESTRATO"].mean(), 2)

dfs

{'Surco': 4.59,
 'Miraflores': 5.0,
 'San Isidro': 5.0,
 'La Molina': 4.76,
 'San Borja': 4.95,
 'Surquillo': 4.49,
 'Cercado De Lima': 3.21,
 'San Miguel': 4.66,
 'Jesus Maria': 5.0,
 'Chorrillos': 2.71,
 'Barranco': 4.11,
 'Lince': 4.27,
 'Los Olivos': 3.22,
 'Pueblo Libre': 4.4,
 'Magdalena Del Mar': 5.0,
 'Ate': 1.51,
 'San Martin': 2.8,
 'Lurigancho': 1.46,
 'Carabayllo': 1.9,
 'Callao': 3.13,
 'La Victoria': 3.27,
 'Comas': 2.36}

In [286]:
df['ranking'] = df['district'].map(dfs)
df

,title,address,price,area,bath,room,property_type,description,year,parking,...,near_avenue,security,elevator,rest_area,pool,web,clean_address,dept,district,ranking
0,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",120375,45,2,3,Departamento,Mozz se deriva de la palabra en italiano mozio...,2023,1,...,0,0,0,1,0,Urbania,calle coronel odriozola 128 san isidrolima lima,Lima,San Isidro,5.00
1,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",119277,45,2,3,Departamento,Mozz se deriva de la palabra en italiano mozio...,2023,1,...,0,0,0,1,0,Urbania,calle coronel odriozola 128 san isidrolima lima,Lima,San Isidro,5.00
2,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",162297,60,2,2,Departamento,Mozz se deriva de la palabra en italiano mozio...,2023,1,...,0,0,0,1,0,Urbania,calle coronel odriozola 128 san isidrolima lima,Lima,San Isidro,5.00
3,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",157815,60,2,2,Departamento,Mozz se deriva de la palabra en italiano mozio...,2023,1,...,0,0,0,1,0,Urbania,calle coronel odriozola 128 san isidrolima lima,Lima,San Isidro,5.00
4,Proyecto inmobilario en venta en Jirón Batalló...,"Jirón Batallón Callao Norte , Santa TeresaSan...",239998,110,3,3,Departamento,Vive exclusivo en Surco | Vive en Libitum | En...,2023,0,...,0,0,0,0,0,Urbania,jiron batallon callao norte santa teresasant...,Lima,Callao,3.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26891,VENTA DEPARTAMENTO JESUS MARIA,"Estados Unidos 135, Jesús María, Perú Jesús M...",143000,80,2,3,Departamento,"En venta, espectacular departamento en Jesús M...",2019,1,...,1,0,1,1,0,Trovit,estados unidos 135 jesus maria peru jesus mar...,Lima,Jesus Maria,5.00
26892,"DEPARTAMENTO MODERNO ,EN JESUS MARIA","Jirón Inca Ripac 197, Jesús María, Perú Jesús...",175000,108,2,3,Departamento,LINDO DEPARTAMENTO CON EXCELENTE VISTA Y UBICA...,2013,1,...,1,0,0,0,0,Trovit,jiron inca ripac 197 jesus maria peru jesus m...,Lima,Jesus Maria,5.00
26905,🚨🚨VENDO DEPARTAMENTO POR DEBAJO DEL PRECIO DE ...,"San Juan de Lurigancho 15427, Perú San Juan d...",85000,157,3,4,Departamento,🚨🚨APROVECHA ESTA 🔥🔥EXCELENTE OPORTUNIDAD DE IN...,2008,0,...,1,0,0,1,0,Trovit,san juan de lurigancho 15427 peru san juan de...,Lima,Lurigancho,1.46
26906,ATE LÍMITE LA MOLINA - VENTA DEPARTAMENTO DE E...,"Parque III - Los Gladiolos, Terreno, Ate, Perú...",104000,78,2,3,Departamento,* VENTA DEPARTAMENTO DE ESTRENO *,0,1,...,1,0,0,0,0,Trovit,parque iii los gladiolos terreno ate peru at...,Lima,Ate,1.51


In [288]:
with open('dataset/estratos.pkl', 'wb') as archivo:
    pickle.dump(dfs, archivo)